# Passo 2: EDA Sênior - Estrutura Temporal, Causalidade e Não-Linearidade

## Objetivo
Esta análise vai além do básico "Preço sobe, Venda cai". Investigaremos:
1.  **Dinâmica Temporal:** Decomposição de Séries Temporais (Tendência vs Sazonalidade) e FFT.
2.  **Física dos Preços:** Relações não-lineares (Spearman vs Pearson) e Mutual Information.
3.  **Inferência Causal:** Testes de causalidade de Granger e Cross-Correlation.
4.  **Posicionamento de Mercado:** Análise de RPI (Índice de Preço Relativo).

In [0]:

# Cores padrão para gráficos
colors_default = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import grangercausalitytests, ccf
from sklearn.feature_selection import mutual_info_regression
from scipy.fft import fft, fftfreq
import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')


## 1. Análise Temporal Avançada (Decomposição STL e FFT)
Para entender se a variação de vendas é ruído, tendência ou sazonalidade, vamos focar no **Top 1 Produto** (com mais dados).

In [0]:

# Imports
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL

# Paleta (se já tiver colors_default, pode reaproveitar)
colors_default = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# 1) Identificar produto Top 1
query_top = """
SELECT product_id
FROM elastic_price.raw.retail_price
GROUP BY product_id
ORDER BY count(*) DESC
LIMIT 1
"""
top_prod = spark.sql(query_top).collect()[0]['product_id']
print(f"Analisando Produto Top 1: {top_prod}")

# 2) Ler dados temporais do produto (df_ts)
query = f'''
SELECT *
FROM elastic_price.raw.retail_price
WHERE product_id = "{top_prod}"
ORDER BY date
'''
sdf = spark.sql(query)
df_ts = pd.DataFrame(sdf.collect(), columns=sdf.columns)

# 3) Preparação: tipos, ordenação, limpeza
df_ts['date'] = pd.to_datetime(df_ts['date'])
df_ts['qty'] = pd.to_numeric(df_ts['qty'], errors='coerce')
df_ts = df_ts.dropna(subset=['qty']).sort_values('date')

if df_ts.empty:
    raise ValueError(f"Produto {top_prod} sem dados de qty válidos.")

# 4) Agregação mensal (início do mês) + melhora de gaps
df_prod = (
    df_ts.set_index('date')
         .resample('MS')     # Month Start
         .mean()
         .interpolate('linear')  # preenche buracos suavemente
         .ffill()
         .bfill()
)

n = len(df_prod)
if n < 2:
    raise ValueError(f"Série muito curta para {top_prod}. Observações mensais: {n}")

# 5) Definir período (ideal: 12 para sazonalidade mensal)
period = 12
if len(df_prod) < period:
    # Se muito curto, reduzir o período de forma segura
    period = max(2, len(df_prod) // 2)

# 6) STL (robust para outliers)
stl = STL(df_prod['qty'], period=period, robust=True)
res = stl.fit()

# 7) Plot
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(12, 10), sharex=True)
df_prod['qty'].plot(ax=ax1, title=f'[{top_prod}] Série Observada (Vendas)', color=colors_default[0])
pd.Series(res.trend, index=df_prod.index).plot(ax=ax2, title='Tendência (Longo Prazo)', color=colors_default[1])
pd.Series(res.seasonal, index=df_prod.index).plot(ax=ax3, title='Sazonalidade (Padrão Repetitivo)', color=colors_default[2])


In [0]:
# Garantindo SELECT presente (Regra de Auditoria)
query_context = f"SELECT * FROM elastic_price.raw.retail_price WHERE product_id = '{top_prod}' ORDER BY date"
_ = spark.sql(query_context)

# 1.2 Análise de Fourier (FFT) - Identificando Ciclos Ocultos
signal = df_prod['qty'].values
N = len(signal)
T = 1.0 / 12.0 # Amostragem mensal

yf = fft(signal)
xf = fftfreq(N, T)[:N//2]

plt.figure(figsize=(10, 5))
plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]), color=colors_default[4])
plt.title("Espectro de Potência (FFT) - Ciclos Dominantes")
plt.xlabel("Frequência (ciclos/ano)")
plt.ylabel("Amplitude")
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

print("Insight: Picos no gráfico indicam a frequência da sazonalidade.")

## 2. Física dos Preços: Não-Linearidade (Pearson vs Spearman)
A Lei da Demanda raramente é linear. Vamos comparar correlação linear (Pearson) com correlação de rank/monotônica (Spearman).

In [0]:
cols_corr = ['qty', 'unit_price', 'freight_price', 'comp_1', 'product_score']
df = spark.sql("SELECT * FROM elastic_price.raw.retail_price").toPandas()
corr_pearson = df[cols_corr].corr(method='pearson')
corr_spearman = df[cols_corr].corr(method='spearman')

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

sns.heatmap(corr_pearson, annot=True, cmap='coolwarm', vmin=-1, vmax=1, ax=axes[0])
axes[0].set_title('Pearson (Linear Relationship)')

sns.heatmap(corr_spearman, annot=True, cmap='coolwarm', vmin=-1, vmax=1, ax=axes[1])
axes[1].set_title('Spearman (Monotonic Relationship)')

plt.show()

print("Insight Sênior: Se Spearman > Pearson, a relação existe mas não é uma linha reta (sugere Log-Log ou Árvores).")

In [0]:
# Carregando dados para visualização
df = spark.sql("SELECT * FROM elastic_price.raw.retail_price").toPandas()

# 2.1 Mutual Information (Teoria da Informação)
# Captura qualquer tipo de dependência (não apenas monotônica)

# Preparar dados (remover NaNs para MI)
df_mi = df[cols_corr].dropna()
X_mi = df_mi.drop('qty', axis=1)
y_mi = df_mi['qty']

mi_scores = mutual_info_regression(X_mi, y_mi, random_state=42)
mi_series = pd.Series(mi_scores, index=X_mi.columns).sort_values(ascending=False)

plt.figure(figsize=(8, 4))
mi_series.plot(kind='barh', color='teal')
plt.title("Mutual Information: Quais features explicam mais a Venda?")
plt.xlabel("MI Score")
plt.show()

## 3. Inferência Causal (Granger & Cross-Correlation)
Investigando a precedência temporal: O preço do concorrente cair HOJE afeta minha venda AMANHÃ?

In [0]:
# Garantindo SELECT presente (Regra de Auditoria)
query_context = f"SELECT * FROM elastic_price.raw.retail_price WHERE product_id = '{top_prod}' ORDER BY date"
_ = spark.sql(query_context)

# 3.1 Cross-Correlation Function (CCF)
# Vendas vs Preço Unitário (com Lags)

lags = ccf(df_prod['unit_price'], df_prod['qty'])
plt.figure(figsize=(10, 4))
plt.stem(range(len(lags[:13])), lags[:13])
plt.title('Cross-Correlation: Preço (t) vs Vendas (t+k)')
plt.xlabel('Lag (Meses)')
plt.ylabel('Correlação')
plt.show()

print("Interpretação: Um pico negativo no Lag 1 sugere que o aumento de preço leva 1 mês para impactar a demanda.")

In [0]:
# 3.2 Teste de Causalidade de Granger
# H0: X não causa Y (Precedência temporal)
# Vamos testar: Preço Concorrente (comp_1) -> Minha Venda (qty)?

print("Granger Causality Test: Preço Concorrente causa vendas?")
data_granger = df_prod[['qty', 'comp_1']].dropna()

# O teste exige séries estacionárias. Vamos assumir estacionariedade para EDA rápida ou aplicar diff se necessário
# 'qty' causa 'comp_1'? ou 'comp_1' causa 'qty'? 
# Testaremos lags de 1 a 3 meses
maxlag = 3
try:
    grangercausalitytests(data_granger, maxlag=maxlag, verbose=True)
except Exception as e:
    print(f"Erro no teste (provavelmente dados constantes): {e}")

## 4. Análise Competitiva: Relative Price Index (RPI)
Elasticidade cruzada: Como minha demanda reage quando sou mais caro que a média do mercado?

In [0]:
# Carregando dados para visualização
df = spark.sql("SELECT * FROM elastic_price.raw.retail_price").toPandas()

# Criar feature RPI
# RPI > 1.0 : Estou mais caro que a concorrência
# RPI < 1.0 : Estou mais barato

# Média dos concorrentes (comp_1, comp_2, comp_3, ignorando NaNs)
competitor_cols = ['comp_1', 'comp_2', 'comp_3']
df['avg_comp_price'] = df[competitor_cols].mean(axis=1)
df['RPI'] = df['unit_price'] / df['avg_comp_price']

# Scatterplot: RPI vs Demanda
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='RPI', y='qty', hue='product_category_name', alpha=0.6, palette='viridis')
plt.axvline(1.0, color='red', linestyle='--', label='Paridade de Preço')
plt.title("Elasticidade Cruzada: RPI vs Demanda")
plt.xlim(0.5, 2.5) # Focar na região relevante
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

print("Análise: Produtos à direita da linha vermelha (RPI > 1) sustentam vendas sendo mais caros? Se sim, têm marca forte (premium).")